## これは各アルゴリズムで区間の共通箇所を算出するパターン

In [1]:
import numpy as np
from sklearn.linear_model import Lasso
from sicore import polytope_to_interval, intersection

In [5]:
def convert_to_nested_list(interval):
    if isinstance(interval[0], list):  # 既に二重のリストである場合
        return interval
    else:  # 単一のリストである場合
        return [interval]

def interval_disassembly(interval1,interval2):
    interval_list = []
    i, j = 0, 0

    while i < len(interval1) and j < len(interval2):
        interval1 = convert_to_nested_list(interval1)
        interval2 = convert_to_nested_list(interval2)
        
        L1, U1 = interval1[i]
        L2, U2 = interval2[j]

        # インターバルが重なっている場合
        if U2 > L1 and U1 > L2:
            new_L = max(L1,L2)
            new_U = min(U1,U2)
            interval_list.append([new_L,new_U])

        # 次のインターバルに進みます
        if U1 < U2:
            i += 1
        else:
            j += 1
    
    return interval_list

# MS SI

In [ ]:
def ms_si(a,b,z,X,A,O,Inter,k):

    # yzの作成
    yz_flatten = a + b * z
    yz = yz_flatten.reshape(-1,1)

    # 外れ値の除去(X,y,a,bに対して)
    X = np.delete(X,[O],0)
    yz = np.delete(yz,[O]).reshape(-1,1)

    a = np.delete(a,[O])
    b = np.delete(b,[O])

    # 特徴の除去
    X = X[:,A]

    # Marginal Screening
    XTyz_abs = np.abs(X.T @ yz).flatten()
    sort_XTyz_abs = np.argsort(XTyz_abs)[::-1]

    Az = sort_XTyz_abs[:k]
    Acz = sort_XTyz_abs[k:]

    # 切断区間算出
    list_u = []
    list_v = []

    for i in Az:
        xj = X[:,i]
        sj = np.sign(np.dot(xj.T,yz))

        e1 = sj * np.dot(xj.T,a)
        e2 = sj * np.dot(xj.T,b)

        for k in Acz:
            xk = X[:,k]
            
            e3 = np.dot(xk.T,a)
            e4 = np.dot(xk.T,b)

            e5 = -np.dot(xk.T,a)
            e6 = -np.dot(xk.T,b)

            list_u.append(e4 - e2)
            list_u.append(e6 - e2)

            list_v.append(e1 - e3)
            list_v.append(e1 - e5)
    
    nu_plus = np.Inf
    nu_minus = np.NINF

    for left,right in zip(list_u,list_v):
        if np.around(left, 5) == 0:
            if right <= 0:
                raise Exception("エラー: 無効な条件です。")
            continue

        temp = right / left

        if left > 0: #論文の条件式より
            nu_plus = min(temp,nu_plus) 
        else:
            nu_minus = max(temp,nu_minus)
    
    assert nu_minus < nu_plus

    inter_z = [nu_minus,nu_plus]
    inter_z = interval_disassembly(inter_z,Inter)

    # 元の特徴に基づいた選択結果
    Az = [A[i] for i in Az]

    return Az,inter_z

# Lasso SI

In [ ]:
def lasso_si(a,b,z,X,A,O,Inter,lamda):

    # yzの作成
    yz_flatten = a + b * z
    yz = yz_flatten.reshape(-1,1)

    # 外れ値の除去(X,y,a,bに対して)
    X = np.delete(X,[O],0)
    yz = np.delete(yz,[O]).reshape(-1,1)

    a = np.delete(a,[O])
    b = np.delete(b,[O])

    # 特徴の除去
    X = X[:,A]

    # lasso
    clf = Lasso(alpha=lamda,fit_intercept=False,max_iter=5000,tol=1e-10)
    clf.fit(X,yz)
    coef = clf.coef_

    # lassoによる結果(インデックス表示)
    Az = np.where(coef != 0)[0].tolist() #coefが0でないものをリストとして表示
    Acz = [i for i in X.shape[1] if i not in Az]
    s = np.sign(coef[Az])

    # XA,XAcの算出
    XA = X[:,Az]
    XAc = X[:,Acz]

    # 切断区間
    lasso_condition = []
    Pm = XA @ np.linalg.inv(XA.T @ XA) @ XA.T
    XA_plus = np.linalg.inv(XA.T @ XA) @ XA.T

    A0_plus = 1 / (lamda*X.shape[0]) * (XAc.T @ (np.identity(X.shape[0]) - Pm) ) 
    A0_minus = 1 / (lamda*X.shape[0]) * (-1 * XAc.T @ (np.identity(X.shape[0]) - Pm))
    b0_plus = np.ones(XAc.shape[1]) - XAc.T @ XA_plus.T @ s
    b0_minus = np.ones(XAc.shape[1]) + XAc.T @ XA_plus.T @ s

    A1 = -1 * np.diag(s) @ np.linalg.inv(XA.T @ XA) @ XA.T
    b1 = -1 * (lamda*X.shape[0]) * np.diag(s) @ np.linalg.inv(XA.T @ XA) @ s

    lasso_condition = [[A0_plus,b0_plus],[A0_minus,b0_minus],[A1,b1]]

    list_u = []
    list_v = []

    nu_plus = np.Inf
    nu_minus = np.NINF

    for Aj,bj in lasso_condition:
        uj = np.dot(Aj,b).reshape(-1).tolist()
        vj = (bj - np.dot(Aj, a)).reshape(-1).tolist()
        list_u.extend(uj)
        list_v.extend(vj)
    
    for left,right in zip(list_u,list_v):

        if np.round(left,5) == 0:
            if right <= 0:
                raise Exception("エラー: 無効な条件です。")
            continue

        temp = right / left

        if left > 0:
            nu_plus = min(temp,nu_plus)
        else:
            nu_minus = max(temp,nu_minus)
    
    assert nu_minus < nu_plus

    inter_z = [nu_minus,nu_plus]
    inter_z = interval_disassembly(inter_z,Inter)

    # 元の特徴に基づいた結果
    Az = [A[i] for i in Az]

    return Az,inter_z

# SFS SI

In [ ]:
def sfs_si(a,b,z,X,A,O,Inter,k):
    
    # yzの作成
    yz_flatten = a + b * z
    yz = yz_flatten.reshape(-1,1)

    # 外れ値の除去(X,y,a,bに対して)
    X = np.delete(X,[O],0)
    yz = np.delete(yz,[O]).reshape(-1,1)

    a = np.delete(a,[O])
    b = np.delete(b,[O])

    # 特徴の除去
    X = X[:,A]

    # sfs
    Az = []
    Acz = list(range(X.shape[1]))
    s = []

    for i in range(k):
        XA = X[:,Az]
        r = yz - XA @ np.linalg.pinv(XA.T @ XA) @ XA.T @ yz
        correlation = X[:,Acz].T @ r 

        index = np.argmax(np.abs(correlation)) #何番目の要素が最大か？

        s.append(np.sign(correlation[index]))

        Az.append(Acz[index])
        Acz.remove(Acz[index])

    # 切断区間
    list_u = []
    list_v = []
    Acz = list(range(X.shape[1])) # 選択されなかった特徴，最初は全特徴を含む

    for i in range(k):
        XA = X[:,Az[0:i]] # 残差r_t 計算用
        x_jt = X[:,Az[i]] # 選択イベント x_\hat{j}_{t} 
        s_t = s[i] # \hat{s}_{t} 
        F = np.identity(X.shape[0]) - XA @ np.linalg.pinv(XA.T @ XA) @ XA.T
        Acz.remove(Az[i])

        for j in Acz:
            x_j = X[:,j]

            u1 = np.dot((x_j - s_t * x_jt).T,np.dot(F,b))
            v1 = np.dot(-(x_j - s_t * x_jt).T,np.dot(F,a))
            u2 = np.dot((- x_j - s_t * x_jt).T,np.dot(F,b))
            v2 = np.dot( -(- x_j - s_t * x_jt).T,np.dot(F,a))

            list_u.append(u1)
            list_u.append(u2)
            list_v.append(v1)
            list_v.append(v2)
    
    nu_plus = np.Inf
    nu_minus = np.NINF

    for m in range(len(list_u)):
        left = list_u[m]
        right = list_v[m]

        if np.around(left, 5) == 0:
            if right <= 0:
                print("ERROR")
                
            continue

        temp = right / left

        if left > 0: #論文の条件式より
            nu_plus = min(temp,nu_plus) 
        else:
            nu_minus = max(temp,nu_minus)
    
    assert nu_minus < nu_plus

    inter_z = [nu_minus,nu_plus]
    inter_z = interval_disassembly(inter_z,Inter)

    # 元の特徴に基づいた結果
    Az = [A[i] for i in Az]

    return Az,inter_z

In [39]:
# sfs
X = np.random.randn(100,10)
y = np.random.randn(100)

a = [0,2,5,7,9]
k = 3
X = X[:,a]

A = []
s = []
Ac = list(range(X.shape[1]))

for i in range(k):
    XA = X[:,A]
    r = y - XA @ np.linalg.pinv(XA.T @ XA) @ XA.T @ y
    correlation = X[:,Ac].T @ r 

    index = np.argmax(np.abs(correlation)) #何番目の要素が最大か？

    s.append(np.sign(correlation[index]))

    A.append(Ac[index])
    Ac.remove(Ac[index])
print(A)


[3, 0, 1]


# Cook SI

In [ ]:
def cook_si(a,b,z,X,A,O,Inter,lamda):

    # yzの作成
    yz_flatten = a + b * z
    yz = yz_flatten.reshape(-1,1)

    # 最後に最終的に得られる外れ値集合を求めるのに使用する
    num_data = list(range(X.shape[0]))
    num_outlier_data = [x for x in num_data if x not in O]

    # 外れ値の除去(X,y,a,bに対して)
    X = np.delete(X,[O],0)
    yz = np.delete(yz,[O]).reshape(-1,1)

    a = np.delete(a,[O])
    b = np.delete(b,[O])

    # 特徴の除去
    X = X[:,A]

    # cook's distance
    non_outlier = []
    outlier = []
    n,p = X.shape

    hat_matrix =  X @ np.linalg.inv(X.T @ X) @ X.T
    Px = np.identity(n) - hat_matrix
    threads = lamda / n #しきい値の設定

    # 外れ値の除去
    for i in range(n):
        ej = np.zeros((n,1))
        ej[i] = 1
        hi = hat_matrix[i][i] #Pxの対角成分
        Di_1 = (y.T @ (Px @ ej @ ej.T @ Px) @ y) / (y.T @ Px @ y) # Diの1項目
        Di_2 = ((n - p) * hi) / (p * (1 - hi)**2) # Diの2項目
        Di = Di_1 * Di_2

        if Di < threads:
            non_outlier.append(i)
        else:
            outlier.append(i)
    
    # 切断区間
    inter_z = [-np.inf, np.inf]
    B = np.zeros(n)
    C = 0 

    for i in range(n):

        ej = np.zeros((n,1))
        ej[i] = 1
        hi = hat_matrix[i][i]
        H_1 = ((n - p) * hi) * Px @ ej @ ej.T @ Px
        H_2 = ((lamda * p * (1 - hi)**2) / n ) * Px
        H = H_1 - H_2

        if i in outlier:
            H = -H

        intervals = polytope_to_interval(a,b,H,B,C)
        inter_z = intersection(intervals,inter_z)
    
    inter_z = interval_disassembly(inter_z,Inter)
    
    # 元の特徴に基づいた結果
    outlier2 = [num_outlier_data[i] for i in outlier]
    Oz = O + outlier2
    
    return Oz,inter_z

## 2連続で外れ値除去する時

In [78]:
import outlier_removel
X = list(range(1,41))
y = list(range(1,11))
outlier = [0,4,6,8]
y = np.array(y).reshape(10,)
X = np.array(X).reshape(10,4)
X = np.delete(X,[outlier],0)
y = np.delete(y,[outlier]).reshape(-1,1)

print(X,y)
non_outlier,outlier = outlier_removel.cook_distance(X,y,0.3)
print(outlier)

[[ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]
 [21 22 23 24]
 [29 30 31 32]
 [37 38 39 40]] [[ 2]
 [ 3]
 [ 4]
 [ 6]
 [ 8]
 [10]]


LinAlgError: Singular matrix

In [97]:
import outlier_removel
np.random.seed(0)
data_list = list(range(10))
print(data_list)
X = np.random.randn(10,5)
y = np.random.randn(10,)
print(y)
outlier = [0,1,2]
data_outlier = [x for x in data_list if x not in outlier]
print(data_outlier)
X = np.delete(X,[outlier],0)
y = np.delete(y,[outlier]).reshape(-1,)
print(y)
non_outlier,outlier = outlier_removel.cook_distance(X,y,3)
print(outlier)
outlier_shin = [x for x in data_outlier if x in outlier]
print(data_outlier)
outlier_sin = [data_outlier[i] for i in outlier]
print(outlier_sin)
print(outlier_shin)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[-0.89546656  0.3869025  -0.51080514 -1.18063218 -0.02818223  0.42833187
  0.06651722  0.3024719  -0.63432209 -0.36274117]
[3, 4, 5, 6, 7, 8, 9]
[-1.18063218 -0.02818223  0.42833187  0.06651722  0.3024719  -0.63432209
 -0.36274117]
[0, 3, 4, 5]
[3, 4, 5, 6, 7, 8, 9]
[3, 6, 7, 8]
[3, 4, 5]


In [102]:
import outlier_removel
np.random.seed(0)
data_list = list(range(10))
print(data_list)
X = np.random.randn(10,5)
y = np.random.randn(10,)
print(y)
O = [0,1,2]
# データから外れ値を除いたインデックスを取得
data_outlier = [x for x in data_list if x not in O]
print(data_outlier)
X = np.delete(X,[O],0)
y = np.delete(y,[O]).reshape(-1,)
print(y)
non_outlier,outlier = outlier_removel.cook_distance(X,y,3)
print(outlier)
# data_outlierから外れ値に該当するインデックスを取得
outlier_sin = [data_outlier[i] for i in outlier]
# 最終的な外れ値集合を取得
O = O + outlier_sin
print(outlier_sin)
print(O)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[-0.89546656  0.3869025  -0.51080514 -1.18063218 -0.02818223  0.42833187
  0.06651722  0.3024719  -0.63432209 -0.36274117]
[3, 4, 5, 6, 7, 8, 9]
[-1.18063218 -0.02818223  0.42833187  0.06651722  0.3024719  -0.63432209
 -0.36274117]
[0, 3, 4, 5]
[3, 6, 7, 8]
[0, 1, 2, 3, 6, 7, 8]


## 最終盤

In [103]:
import outlier_removel
np.random.seed(0)
data_list = list(range(10))
X = np.random.randn(10,5)
y = np.random.randn(10,)
# 最初の外れ値集合
O = [0,1,2]
# データから外れ値を除いたインデックスを取得
data_outlier = [x for x in data_list if x not in O]

# X,yから外れ値に該当するデータを削除
X = np.delete(X,[O],0)
y = np.delete(y,[O]).reshape(-1,)

# クックの距離による外れ値除去
non_outlier,outlier = outlier_removel.cook_distance(X,y,3)

# data_outlierから外れ値に該当するインデックスを取得
outlier_sin = [data_outlier[i] for i in outlier]
# 最終的な外れ値集合を取得
O = O + outlier_sin

print(outlier_sin)
print(O)

[3, 6, 7, 8]
[0, 1, 2, 3, 6, 7, 8]


# DFFITS SI

In [ ]:
def dffits_si(a,b,z,X,A,O,Inter,lamda):

    # yzの作成
    yz_flatten = a + b * z
    yz = yz_flatten.reshape(-1,1)

    # 最後に最終的に得られる外れ値集合を求めるのに使用する
    num_data = list(range(X.shape[0]))
    num_outlier_data = [x for x in num_data if x not in O]

    # 外れ値の除去(X,y,a,bに対して)
    X = np.delete(X,[O],0)
    yz = np.delete(yz,[O]).reshape(-1,1)

    a = np.delete(a,[O])
    b = np.delete(b,[O])

    # 特徴の除去
    X = X[:,A]

    # DFFITS
    non_outlier = []
    outlier = []
    n,p = X.shape

    hat_matrix =  X @ np.linalg.inv(X.T @ X) @ X.T
    Px = np.identity(n) - hat_matrix
    threads = (lamda * p) / (n - p) #しきい値の設定

    # 外れ値の除去
    for i in range(n):
        ej = np.zeros((n,1))
        ej[i] = 1
        hi = hat_matrix[i][i] #Pxの対角成分
        DFFITSi_1 = np.sqrt(hi * (n - p - 1)) / (1 - hi) # DFFITSの片側
        DFFITSi_2_denominator = y.T @ Px @ y - ((y.T @ Px @ ej @ ej.T @ Px @ y) / (1 - hi))
        DFFITSi_2 = (ej.T @ Px @ y) / np.sqrt(DFFITSi_2_denominator )
        DFFITSi = DFFITSi_1 * DFFITSi_2

        if DFFITSi**2 < threads:
            non_outlier.append(i)
        else:
            outlier.append(i)
    
    # 切断区間
    inter_z = [-np.inf, np.inf]
    B = np.zeros(n)
    C = 0 

    for i in range(n):
        ej = np.zeros((n,1))
        ej[i] = 1
        hi = hat_matrix[i][i]
        H_1_1 = ((hi * (n - p - 1)) / (1 - hi)**2) + ((lamda * p) / ((n - p)*(1 - hi)))
        H_1 = H_1_1 * Px @ ej @ ej.T @ Px 
        H_2 = ((lamda * p)/(n - p)) * Px
    
        H = H_1 - H_2
        if i in outlier:
            H = - H

        intervals = polytope_to_interval(a,b,H,B,C)
        inter_z = intersection(intervals,inter_z)
    
    inter_z = interval_disassembly(inter_z,Inter)

    # 元の特徴に基づいた結果
    outlier2 = [num_outlier_data[i] for i in outlier]
    Oz = O + outlier2
    
    return Oz,inter_z

# soft-IPOD SI

In [ ]:
# soft-IPODのハイパーパラメータは外部から入力する感じ
def soft_ipod_si(a,b,z,X,A,O,Inter,lamda):
    
    # yzの作成
    yz_flatten = a + b * z
    yz = yz_flatten.reshape(-1,1)

    # 最後に最終的に得られる外れ値集合を求めるのに使用する
    num_data = list(range(X.shape[0]))
    num_outlier_data = [x for x in num_data if x not in O]

    # 外れ値の除去(X,y,a,bに対して)
    X = np.delete(X,[O],0)
    yz = np.delete(yz,[O]).reshape(-1,1)

    a = np.delete(a,[O])
    b = np.delete(b,[O])

    # 特徴の除去
    X = X[:,A]

    # soft-IPODの準備
    n,p = X.shape

    hat_matrix =  X @ np.linalg.inv(X.T @ X) @ X.T
    PXperp = np.identity(n) - hat_matrix
    PXperpy = PXperp @ y

    # soft-IPODの実行
    clf = Lasso(alpha=lamda,fit_intercept=False,max_iter=5000,tol=1e-10)
    clf.fit(PXperp,PXperpy)
    coef = clf.coef_
    outlier = np.where(coef!=0)[0].tolist() #外れ値
    non_outlier = np.where(coef==0)[0].tolist() #非外れ値
    s = np.sign(coef[outlier])

    # 切断区間
    soft_IPOD_condition = []
    # PXperpを計算
    I = np.identity(n)
    X_caron =  I - X @ np.linalg.inv(X.T @ X) @ X.T

    X_caron_M = X_caron[:,non_outlier]
    X_caron_Mc = X_caron[:,outlier]

    PX_caron_Mc_perp = I - X_caron_Mc @ np.linalg.inv(X_caron_Mc.T @ X_caron_Mc) @ X_caron_Mc.T
    X_caron_Mc_plus = X_caron_Mc @ np.linalg.inv(X_caron_Mc.T @ X_caron_Mc)

    # 以下はy_caronに対する係数
    A0_plus = (1 / (lamda * n)) * (X_caron_M.T @ PX_caron_Mc_perp @ X_caron)
    A0_minus = -1 *  (1 / (lamda * n)) * (X_caron_M.T @ PX_caron_Mc_perp @ X_caron)

    b0_plus = np.ones(len(non_outlier)) - X_caron_M.T @ X_caron_Mc_plus @ s
    b0_minus = np.ones(len(non_outlier)) + X_caron_M.T @ X_caron_Mc_plus @ s

    A1 = -1 * np.diag(s) @ np.linalg.inv(X_caron_Mc.T @ X_caron_Mc) @ X_caron_Mc.T @ X_caron
    b1 = -1 * n * lamda * np.diag(s) @ np.linalg.inv(X_caron_Mc.T @ X_caron_Mc) @ s

    soft_IPOD_condition = [[A0_plus,b0_plus],[A0_minus,b0_minus],[A1,b1]]

    list_u = []
    list_v = []

    nu_plus = np.Inf
    nu_minus = np.NINF

    for j in soft_IPOD_condition:
        Aj,bj = j
        uj = ((Aj @ b).reshape(-1)).tolist()
        vj = ((bj - Aj @ a).reshape(-1)).tolist()
        list_u.extend(uj)
        list_v.extend(vj)

    for m in range(len(list_u)):
        left = list_u[m]
        right = list_v[m]

        if np.around(left,5) == 0:
            if right <= 0:
                print("ERROR")
                
            continue

        temp = right / left

        if left > 0:
            nu_plus = min(temp,nu_plus)
        else:
            nu_minus = max(temp,nu_minus)

    assert nu_minus < nu_plus

    inter_z = [nu_minus,nu_plus]
    inter_z = interval_disassembly(inter_z,Inter)

    # 元の特徴に基づいた結果
    outlier2 = [num_outlier_data[i] for i in outlier]
    Oz = O + outlier2

    return Oz,inter_z

# 欠損値補完

## 平均値補完

In [ ]:
def mean_value_imputation(X,y,sigma):

    # データ数の取得
    n = y.shape[0]

    # 欠損箇所を取得
    missing_index = np.where(np.isnan(y))[0]

    # index_random番目の以外のyを取得
    y_delete = np.delete(y, missing_index)

    # 平均値を計算
    y_mean = np.mean(y_delete)

    # 欠損値補完，(100,1)に変換
    y[missing_index] = y_mean
    y = y.reshape((n,1))

    # yの分散共分散行列の変更
    cov = np.identity(n)

    # 該当する箇所の分散共分散を変更していく
    # (1 / (n - 1)) * (y_1 + y_2 + ... + y_n-1)
    each_var_cov_value = sigma**2 / (n - len(missing_index))

    cov[:,missing_index] = each_var_cov_value
    cov[missing_index,:] = each_var_cov_value

    return X,y,cov 

## Hot-deck法  (ユークリッド，マンハッタン，チェビシェフ)

In [ ]:
# ユークリッド距離
def euclidean_imputation(X,y,sigma):

    # 欠損箇所を取得
    missing_index = np.where(np.isnan(y))[0]

    idx_list = []

    for index_random in missing_index:
        # ユークリッド距離の計算，二種類
        X_euclidean = np.sqrt(np.sum((X - X[index_random])**2, axis=1))

        # 欠損箇所を除いて，ユークリッド距離が最小になる
        X_euclidean_deleted = np.delete(X_euclidean, missing_index)
        idx_deleted = np.argmin(X_euclidean_deleted) # missing_indexを除いたindex

        # idx_deletedは欠損値を除いたデータセットに対するものなので、元のデータセットに対する正しいインデックスを得るためには、欠損値のインデックスが現在のインデックスより小さい場合には+1します。
        # idx_deletedを求めた時と同じ条件(欠損箇所を除いた)で，idxを求める
        original_indices = np.arange(len(X_euclidean))  # 元のデータセットのインデックス
        valid_indices = np.delete(original_indices, missing_index)  # 欠損値を除いたインデックス
        idx = valid_indices[idx_deleted]  # 欠損値を除いたインデックスから元のインデックスを取得

        y[index_random] = y[idx]

        idx_list.append(idx)

    y = y.reshape((X.shape[0],1))

    # 6. 分散共分散行列の作成
    cov = np.identity(X.shape[0])

    for index_random,idx in zip(missing_index,idx_list):
        cov[index_random,idx] = sigma**2
        cov[idx,index_random] = sigma**2

    return X,y,cov

# マンハッタン距離
def manhattan_imputation(X,y,sigma):

    # 欠損箇所を取得
    missing_index = np.where(np.isnan(y))[0]

    idx_list = []

    for index_random in missing_index:

        # マンハッタン距離の計算
        X_manhattan = np.sum(np.abs(X - X[index_random]), axis=1)

        # 欠損箇所を除いて，マンハッタン距離が最小になる
        X_manhattan_deleted = np.delete(X_manhattan, missing_index)
        idx_deleted = np.argmin(X_manhattan_deleted) # missing_indexを除いたindex

        # idx_deletedは欠損値を除いたデータセットに対するものなので、元のデータセットに対する正しいインデックスを得るためには、欠損値のインデックスが現在のインデックスより小さい場合には+1します。
        # idx_deletedを求めた時と同じ条件(欠損箇所を除いた)で，idxを求める
        original_indices = np.arange(len(X_manhattan))  # 元のデータセットのインデックス
        valid_indices = np.delete(original_indices, missing_index)  # 欠損値を除いたインデックス
        idx = valid_indices[idx_deleted]  # 欠損値を除いたインデックスから元のインデックスを取得

        y[index_random] = y[idx]

        idx_list.append(idx)

    y = y.reshape((X.shape[0],1))

    # 6. 分散共分散行列の作成
    cov = np.identity(X.shape[0])

    for index_random,idx in zip(missing_index,idx_list):
        cov[index_random,idx] = sigma**2
        cov[idx,index_random] = sigma**2

    return X,y,cov

# チェビシェフの距離
def chebyshev_imputation(X,y,sigma):

    # 欠損箇所を取得
    missing_index = np.where(np.isnan(y))[0]

    idx_list = []

    for index_random in missing_index:
        # チェビシェフの距離を求める
        X_chebyshev = np.max(np.abs(X - X[index_random]), axis=1)

        # 欠損箇所を除いて，チェビシェフの距離が最小になる
        X_chebyshev_deleted = np.delete(X_chebyshev, missing_index)
        idx_deleted = np.argmin(X_chebyshev_deleted) # missing_indexを除いたindex

        # idx_deletedは欠損値を除いたデータセットに対するものなので、元のデータセットに対する正しいインデックスを得るためには、欠損値のインデックスが現在のインデックスより小さい場合には+1します。
        # idx_deletedを求めた時と同じ条件(欠損箇所を除いた)で，idxを求める
        original_indices = np.arange(len(X_chebyshev))  # 元のデータセットのインデックス
        valid_indices = np.delete(original_indices, missing_index)  # 欠損値を除いたインデックス
        idx = valid_indices[idx_deleted]  # 欠損値を除いたインデックスから元のインデックスを取得

        y[index_random] = y[idx]

        idx_list.append(idx)

    y = y.reshape((X.shape[0],1))

    # 6. 分散共分散行列の作成
    cov = np.identity(X.shape[0])

    for index_random,idx in zip(missing_index,idx_list):
            cov[index_random,idx] = sigma**2
            cov[idx,index_random] = sigma**2

    return X,y,cov



## 回帰代入法

In [ ]:
# 確定的
def regression_definite_imputation(X,y,sigma):
    # 欠損箇所を取得
    n = y.shape[0]
    missing_index = np.where(np.isnan(y))[0]

    # missing_index番目の以外のX,yを取得
    X_delete = np.delete(X, missing_index, 0)
    y_delete = np.delete(y, missing_index, 0).reshape(-1,1)

    cov = np.identity(n)

    # 回帰係数の推定
    beta_hat = np.linalg.inv(X_delete.T @ X_delete) @ X_delete.T @ y_delete

    data_list = list(range(n))

    # 各欠損箇所に対して
    for index_random in missing_index:

        # 欠損しているデータを取得
        X_missing = X[index_random]

        # beta_hatにより欠損していた値を補完
        y_new = X_missing @ beta_hat

        # 欠損箇所の補完
        y[index_random] = y_new

        # 分散の計算
        # 列ごとに分散，共分散を計算しているイメージ
        for i in data_list:
            # データの抽出
            each_x = X[i]
    
            # 分散の計算
            var_missing = sigma**2 * each_x @ np.linalg.inv(X_delete.T @ X_delete) @ X_missing.T
            #var_missing = (sigma * each_x.T @ np.linalg.inv(X_delete.T @ X_delete) @ X_missing)[0,0]

            # 分散共分散行列に追加
            cov[i,index_random] = var_missing
            cov[index_random,i] = var_missing

    y = y.reshape((n,1))

    return X,y,cov

# 確率的
def regression_probabilistic_imputation(X,y,sigma):
    # 欠損箇所を取得
    n = y.shape[0]
    missing_index = np.where(np.isnan(y))[0]

    # missing_index番目の以外のX,yを取得
    X_delete = np.delete(X, missing_index, 0)
    y_delete = np.delete(y, missing_index, 0).reshape(-1,1)

    cov = np.identity(n)

    # 回帰係数の推定
    beta_hat = np.linalg.inv(X_delete.T @ X_delete) @ X_delete.T @ y_delete

    data_list = list(range(n))

    # 各欠損箇所に対して
    for index_random in missing_index:

        # 欠損しているデータを取得
        X_missing = X[index_random]

        # beta_hatにより欠損していた値を補完
        rng = np.random.default_rng()
        noise = rng.standard_normal()

        y_new = X_missing @ beta_hat + noise

        # 欠損箇所の補完
        y[index_random] = y_new

        # 分散の計算
        # 列ごとに分散，共分散を計算しているイメージ
        for i in data_list:
            # データの抽出
            each_x = X[i]

            if i == index_random or i in missing_index:
                var_missing = sigma**2 * each_x @ np.linalg.inv(X_delete.T @ X_delete) @ X_missing.T + 1
                cov[i,index_random] = var_missing
                cov[index_random,i] = var_missing
            
            else:
                var_missing = sigma**2 * each_x @ np.linalg.inv(X_delete.T @ X_delete) @ X_missing.T
                cov[i,index_random] = var_missing
                cov[index_random,i] = var_missing

            #var_missing = (sigma * each_x.T @ np.linalg.inv(X_delete.T @ X_delete) @ X_missing)[0,0]

    y = y.reshape((n,1))

    return X,y,cov